## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")

2022-11-13 11:38:09.797724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=["EIN","NAME"])


In [3]:
# Generate our categorical variable lists
cat = df.dtypes[df.dtypes == "object"].index.tolist()
cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [4]:
# Convert categorical data to numeric with `pd.get_dummies`
new_df = pd.get_dummies(data=df, columns=['APPLICATION_TYPE','INCOME_AMT', 'AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS'])
new_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
34296,1,5000,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [5]:
# Split our preprocessed data into our features and target arrays
# Remove IS_SUCCESSFUL target from features data
y = new_df.IS_SUCCESSFUL.values
X = new_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [7]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 14

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#3rd_layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

2022-11-13 11:38:27.633434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1170      
                                                                 
 dense_1 (Dense)             (None, 12)                132       
                                                                 
 dense_2 (Dense)             (None, 14)                182       
                                                                 
 dense_3 (Dense)             (None, 1)                 15        
                                                                 
Total params: 1,499
Trainable params: 1,499
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.5996 - accuracy: 0.7020
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5600 - accuracy: 0.7280
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7290
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5488 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5464 - accuracy: 0.7335
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5446 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5342 - accuracy: 0.7412
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5341 - accuracy: 0.7400
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5342 - accuracy: 0.7411
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7392
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5338 - accuracy: 0.7409
Epoch 86/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5340 - accuracy: 0.7400
Epoch 87/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5339 - accuracy: 0.7402
Epoch 88/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5340 - accuracy: 0.7411
Epoch 89/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5339 - accuracy: 0.7415
Epoch 90/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5336 - accuracy: 

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5554 - accuracy: 0.7272 - 1s/epoch - 5ms/step
Loss: 0.5553853511810303, Accuracy: 0.7272303104400635


In [11]:
# Export our model to HDF5 file
from tensorflow.keras.models import load_model
model=nn
model.save('opt_model3.HDF5')

INFO:tensorflow:Assets written to: opt_model3.HDF5/assets
